<a href="https://colab.research.google.com/github/manju1201/IR_Project_Ranked_Retrieval/blob/main/IR_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [3]:
pip install wordninja

     |████████████████████████████████| 542kB 12.8MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp36-none-any.whl size=541553 sha256=b0ffe3a9be8559e8d31fb4ced9cc562e7a5418cc05a5d915257a7987287f0d39
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


In [4]:
import nltk
import numpy as np
import pandas as pd
import re

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import wordninja

In [5]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Removes Html codes

In [6]:
def remove_htmlcodes(document):
    replacement = {
                    "&ampnbsp": ' ',
                    "&ampamp": '&',
                    "&ampquot": '\'',
                    "&ampldquo": '\"',
                    "&amprdquo": '\"',
                    "&amplsquo": '\'',
                    "&amprsquo": '\'',
                    "&amphellip": '...',
                    "&ampndash": '-',
                    "&ampmdash": '-'
                  }
    for str in replacement:
        document = document.replace(str, replacement[str])
        
    return document

**Preprocessing**

*   Parts of Speech tags,
*   lemmatization,
*   Tokenize, 
*   remove stopwords


In [7]:
def get_wordnet_pos(word):
    tag=nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={"J": wordnet.ADJ, 
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemma_stop(str):
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer('\w+|\$]\d\[+|\S+,-')
    tokenized = tokenizer.tokenize(str)
    lemmatized = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in tokenized]
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in lemmatized if w.lower() not in stop_words]
    after_lemma_stop = ' '.join(w for w in filtered_sentence)
    return filtered_sentence

In [10]:
data = pd.read_csv("/content/drive/MyDrive/IR_domain_custom_dataset_processed.csv")
for column in data:
    if data[column].isnull().any():
        print('{0} has {1} null values'.format(column, data[column].isnull().sum()))

Text has 3 null values
domain has 53555 null values
word_count has 53558 null values


Droping Null values

In [11]:
data = data.dropna()

In [12]:
for column in data:
    if data[column].isnull().any():
        print('{0} has {1} null values'.format(column, data[column].isnull().sum()))
data['word_count'].count()

99998

In [13]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Text,domain,word_count
0,0,1,buy new socceroos jersey think bit excessive f...,1.0,43.0
1,1,6,england trip feel like dream assume lose germa...,1.0,32.0
2,2,7,<tag_1> not rate henderson england player stop...,1.0,30.0
3,3,9,awesome <tag_1> morning talk <tag_2> <tag_3> &...,1.0,46.0
4,4,13,utterly shocked hear leroy sane axe germany sq...,1.0,45.0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99998 entries, 0 to 153555
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    99998 non-null  int64  
 1   Unnamed: 0.1  99998 non-null  object 
 2   Text          99998 non-null  object 
 3   domain        99998 non-null  float64
 4   word_count    99998 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.6+ MB


In [15]:
data = np.array(data)
np.save("datan",data)
data = np.load("datan.npy",allow_pickle = True)

In [16]:
data

array([[0, '1',
        'buy new socceroos jersey think bit excessive figure wear game game great value worldcuprussia2018 aus # socceroos',
        1.0, 43.0],
       [1, '6',
        'england trip feel like dream assume lose germany quarter penalty thank goodness not happen cheer <tag_1>',
        1.0, 32.0],
       [2, '7',
        '<tag_1> not rate henderson england player stop shit oh solid blue # comeonengland # coyb',
        1.0, 30.0],
       ...,
       [153553, '98920',
        '<tag_1> <tag_2> <tag_3> <tag_4> nerp not buy alias invalidate claim responsibility',
        3.0, 16.0],
       [153554, '98921',
        '<tag_1> oh leaker ok hillary talk wiki leak fbi leak hypocrite',
        3.0, 23.0],
       [153555, '98922',
        '<tag_1> need convince people think great yesterday disturb',
        3.0, 14.0]], dtype=object)

##Creating Doc ID and index

In [ ]:
get_docID = {}
get_index = {}


NN = len(data)
for i in range(0, len(data)) :
    get_docID[i] = data[i][0]
    get_index[data[i][0]] = i
print(get_docID)

In [20]:
def is_not_credible (text):
    match = re.search(r'[!@#?&{}()]', text)
    
    if match:
        return True
    else:
        return False

In [21]:
def scrub_words(text):
    text = re.sub('[!@#?&{}()]', '', text)
    text=re.sub(r'[^\x00-\x7F]'," ",text)
    return text

In [22]:
def clean_document (document_string):
    cleaned_doc = document_string
    for word in document_string.split():
      if is_not_credible(word):
        temp= scrub_words(word)
        split=wordninja.split(temp)
        if len(split)>7:
              cleaned_doc = cleaned_doc.replace(word,'')
    return cleaned_doc

In [23]:
subset = []
counter = 0
for document in data:
    subset.append(document)
    counter += 1
    if counter == NN:
        break
print(len(subset))

99998


In [24]:
from datetime import datetime
def replace_dates(documentString):
    
    regEx = '(([0-9]+(/|\\.|-)[0-9]+(/|\\.|-)[0-9]+)|([0-9]+(/|\\.|-)[0-9]+))'
    iterator = re.finditer(regEx, documentString)
    listOfDates = [(m.start(0), m.end(0)) for m in iterator]
    
    for indices in listOfDates:
        date = documentString[indices[0]:indices[1]]
        tmp = date
        date = date.replace('.', '/')
        date = date.replace('-', '/')
        count = date.count('/')
        newDate = ''
        if count == 2:
            try:
                newDate = datetime.strptime(date, '%m/%d/%Y').strftime('%d %b %Y')
            except ValueError as ve:
                newDate = date
        else:
            try:
                newDate = datetime.strptime(date, '%m/%d').strftime('%d %b')
            except ValueError as ve:
                newDate = date
                
        newDate = newDate.replace(' ', '')
        documentString = documentString.replace(tmp, newDate)
        # print(newDate)
    
    return documentString

In [25]:
import time
from tqdm import tqdm

start = time.time()

contents = []
for document in tqdm(subset):
    # actually modifying the document
    document[2] = remove_htmlcodes(document[2])
    
    # not actually modifying the document
    modifiedContent = replace_dates(document[2])
    modifiedContent = lemma_stop(clean_document(modifiedContent))
    
    # case-folding
    for i in range(len(modifiedContent)):
        modifiedContent[i] = modifiedContent[i].lower()
  
    contents.append(modifiedContent)
    
print(time.time() - start)

100%|██████████| 99998/99998 [05:52<00:00, 283.73it/s]

352.4477972984314


In [27]:
pip install unidecode

     |████████████████████████████████| 245kB 23.4MB/s 


In [28]:
import unidecode
contents_temp = contents


for i in range(NN):
    for j in range(len(contents[i])):
        contents[i][j] = unidecode.unidecode(contents[i][j])

##Trie 

In [29]:
class Node:
    
    def __init__(self):
        
        self.prefixes = 0
        self.words = 0
        # self.children = [None for i in range (128)]
        self.children = {}
        
    def add(self, suffix, currentIndex):

        '''Adds suffix[currenIndex : ] to self'''

        if currentIndex == len(suffix):
            self.words = self.words + 1
            self.prefixes = self.prefixes + 1
        
        else:
            self.prefixes = self.prefixes + 1
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     newNode = Node()
            #     self.children[ord(firstChar)] = newNode
            if ord(firstChar) not in self.children:
                newNode = Node()
                self.children[ord(firstChar)] = newNode
            currentIndex += 1
            self.children[ord(firstChar)].add(suffix, currentIndex)
                
    
    def count_words(self, suffix, currentIndex):

        '''Returns the count of words ending with suffix[currentIndex]'''
        
        if currentIndex == len(suffix):
            return self.words
        
        else:
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     return 0
            if ord(firstChar) not in self.children:
                return 0
            else:
                currentIndex += 1
                return self.children[ord(firstChar)].count_words(suffix, currentIndex)
            
    def count_prefixes(self, suffix, currentIndex):

        '''Returns the count of prefixes ending with suffix[currentIndex]'''
        
        if currentIndex == len(suffix):
            return self.prefixes
        
        else:
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     return 0
            if ord(firstChar) not in self.children:
                return 0
            else:
                currentIndex += 1
                return self.children[ord(firstChar)].count_prefixes(suffix, currentIndex)
            
class CollectionNode(Node):
    
    def __init__(self):
        super(CollectionNode,self).__init__()
        self.documents = set()
        self.titles = set()

    def add_title(self, suffix, currentIndex, docID):

        '''Adds suffix[currenIndex : ] to self and appends docID to self.titles
        at the end of suffix'''

        if currentIndex == len(suffix):
            self.words = self.words + 1
            self.prefixes = self.prefixes + 1
            self.titles.add(docID)

        else:
            self.prefixes = self.prefixes + 1
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     newNode = CollectionNode()
            #     self.children[ord(firstChar)] = newNode
            if ord(firstChar) not in self.children:
                newNode = CollectionNode()
                self.children[ord(firstChar)] = newNode
            currentIndex += 1
            self.children[ord(firstChar)].add_title(suffix, currentIndex, docID)

        
    def add_document(self, suffix, currentIndex, docID):

        '''Adds docID to self.documents at the end of suffix'''
        
        if currentIndex == len(suffix):
            self.words = self.words + 1
            self.prefixes = self.prefixes + 1
            self.documents.add(docID)
        
        else:
            self.prefixes = self.prefixes + 1
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     newNode = CollectionNode()
            #     self.children[ord(firstChar)] = newNode
            if ord(firstChar) not in self.children:
                newNode = CollectionNode()
                self.children[ord(firstChar)] = newNode
            currentIndex += 1
            self.children[ord(firstChar)].add_document(suffix, currentIndex, docID)
            
    def get_doc_list(self, suffix, currentIndex):

        '''Returns list of documents containing suffix'''
        
        if currentIndex == len(suffix):
            return self.documents
        
        else:
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     return []
            if ord(firstChar) not in self.children:
                return []
            else:
                currentIndex += 1
                return self.children[ord(firstChar)].get_doc_list(suffix, currentIndex)

    def get_title_list(self, suffix, currentIndex):

        '''Returns list of titles that contain suffix'''
        
        if currentIndex == len(suffix):
            return self.titles
        
        else:
            firstChar = suffix[currentIndex]
            # if self.children[ord(firstChar)] == None:
            #     return []
            if ord(firstChar) not in self.children:
                return []
            else:
                currentIndex += 1
                return self.children[ord(firstChar)].get_title_list(suffix, currentIndex)

##### Create map from docID of the document to an object of class Node (i.e, the corresponding document trie structure)
#####ex. if the docID of the document is 1, 
#####getReference[1] gives the object which is the trie structure of docID 1

In [30]:
import pickle 

getReference = {}
documentRoot = []
collection = CollectionNode()

for i in range(NN):
    newDocument = Node()
    documentRoot.append(newDocument)
    getReference[get_docID[i]] = newDocument

In [31]:
max_tf = {}

import time
from tqdm import tqdm

start = time.time()
for i in tqdm(range(NN)):
    for w in contents_temp[i]:
        collection.add_document(w, 0, get_docID[i])
        documentRoot[i].add(w, 0)
        if get_docID[i] in max_tf:
            max_tf[get_docID[i]] = max(documentRoot[i].count_words(w, 0), max_tf[get_docID[i]])
        else:
            max_tf[get_docID[i]] = documentRoot[i].count_words(w, 0)
    
        
print(time.time() - start)  

100%|██████████| 99998/99998 [00:54<00:00, 1826.12it/s]

54.76555824279785


In [32]:
import math
import queue

documentLength = {}
N = len(documentRoot)


for i in tqdm(range(len(documentRoot))):
    
    docID = get_docID[i]
    length = 0
    document = documentRoot[i]
    q = queue.Queue()
    q.put([document, ''])
    
    while q.qsize() > 0:

        current = q.get()
        reference = current[0]
        word = current[1]

        if reference.words > 0:
            df = len(collection.get_doc_list(word, 0))
            idf = math.log10(N/df)
            # print(word, reference.words, df)
            length += (reference.words * idf) ** 2

        for i in range(256):
            if reference.children.get(i) is not None:
                new_word = word + chr(i)
                q.put([reference.children[i], new_word])

    # print(length**0.5)
    documentLength[docID] = length**0.5
    
    
    
pickle_out = open("collection.pickle","wb")
pickle.dump(collection,pickle_out)
pickle_out.close()
pickle_out1 = open("documentRoot.pickle","wb")
pickle.dump(getReference,pickle_out1)
pickle_out1.close()
pickle_out2 = open("max_tf.pickle","wb")
pickle.dump(max_tf,pickle_out2)
pickle_out2.close()

100%|██████████| 99998/99998 [07:19<00:00, 227.54it/s]


In [33]:
import sys
print(sys.getsizeof(documentRoot))

824464


In [42]:
query = input()
final_query = replace_dates(query)
final_query = lemma_stop(final_query)
for i in range(len(final_query)):
    final_query[i] = unidecode.unidecode(final_query[i])
    
    # case-folding
    final_query[i] = final_query[i].lower()
print(final_query)
print(len(final_query))

world cup
['world', 'cup']
2


In [43]:
tf_query = {}
for w in final_query:
    if w not in tf_query:
        tf_query[w] = 1
    else:
        tf_query[w] += 1


In [44]:
# scores[i] stores the dot product of the tf-idf score vectors of the query and document of docID i in the corpus
scores = {}
title_score = {}

# N is the total number of documents in the corpus
N = len(documentRoot)

# wordsInDoc[i] is a sorted list of (word, score) tuples where
# score is the tf-idf score for the (word, <ith doc>) pair
wordsInDoc = {}

factor = {}

import math
import bisect

for query_term in tf_query:
    
    docs_having_query_term = collection.get_doc_list(query_term, 0)
    print(docs_having_query_term)
    df = len(docs_having_query_term)
    idf = 0
    
    print('-------------------------------------')
    print('Term in query = ', query_term)
    # print('List of documents with this term=', docs_having_query_term)
    print()
    
    if df == 0:
        idf = 0
    else:
        idf = math.log10(N/df)
        
    docs_having_query_term_in_title = collection.get_title_list(query_term,0)
    print(docs_having_query_term_in_title)
    for docID in docs_having_query_term_in_title:
        if docID in title_score:
            title_score[docID] += idf
        else:
            title_score[docID] = idf
        
    print('df = ',df)
    print('idf = ',idf)
    
    tfidf_query = tf_query[query_term] * idf
        
    for docID in docs_having_query_term:
        # print(docID)
        tf_doc = getReference[docID].count_words(query_term, 0)
        tf_doc = 0.5 + 0.5*tf_doc/max_tf[docID]
        # print('tf for doc',docID,'is',tf_doc)
        # tfidf_doc_query = tf_doc * idf
        # tfidf_doc = 1 + math.log10(tf_doc)
        tfidf_doc = (tf_doc)
        # tfidf_doc_query = (tf_doc)
        
        # print('tfidf for doc',doc,'is',tfidf_doc)
        # print()
        
        if docID not in scores:
            scores[docID] = (tfidf_query * tfidf_doc)
            wordsInDoc[docID] = []
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] = idf
        else:
            scores[docID] += (tfidf_query * tfidf_doc)
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] += idf
print(title_score)
for docID in scores:
    if documentLength[docID] != 0:
#         scores[docID] = scores[docID]/ math.sqrt(documentLength[docID])
        scores[docID] *= factor[docID]
        if docID in title_score:
            scores[docID] *= 1 + title_score[docID]


sorted_scores = sorted(scores.items(), key = lambda kv : kv[1] , reverse = True)

maxshow = min(10, len(scores))

print('\n\n')
print('============================================')

for i in range(maxshow):
    # print(i)
    print()
    docID = sorted_scores[i][0]
    print('doc ID = ', docID)
    cnt = 0
    print('Keywords:')
    print()
    print(subset[get_index[sorted_scores[i][0]]][2])
    print()
    # if sorted_scores[i][0] not in title_score:
    #     print('title score = ',0)
    # else:
    #     print('title score = ',title_score[sorted_scores[i][0]])
    for j in range(len(wordsInDoc[docID])):
        print(wordsInDoc[docID][j][1], wordsInDoc[docID][j][0], end = ' ')
        print(getReference[docID].count_words(wordsInDoc[docID][j][1], 0))
    print()
    print()
    count = 0
    found = 0
    words_before=queue.Queue()
    at_start = 1
    display = ""
    for word in subset[get_index[docID]][2].split():
            
        check_with=replace_dates(word)
        check_with = check_with.lower()
        if len(lemma_stop(check_with)) > 0:
            check_with=lemma_stop(check_with)[0]
        else:
            check_with=word
        
        if check_with == wordsInDoc[docID][0][1]:
            found=1
            
        if found == 1:
            display = display + word + " "
            count += 1
            if count == 50:
                break
        if found == 0:
            words_before.put(word)
            if words_before.qsize()>20:
                remove=words_before.get()
                at_start=0
                
    if not at_start:
        print('...', end = ' ')
    while words_before.qsize() > 0:
        print(words_before.get(), end = ' ')
    print(display, end = ' ')
    print('...', end = ' ')
    print('\n')
    print('tf-idf score=', sorted_scores[i][1])
    print('\n')
    print('============================================')

{32771, 131075, 32775, 8, 11, 65551, 19, 20, 22, 26, 34, 32802, 32806, 65586, 51, 56, 32824, 32828, 63, 32831, 32832, 32837, 70, 72, 32841, 32843, 32849, 32850, 32851, 32852, 65624, 32861, 65629, 101, 32875, 32877, 110, 32880, 114, 32884, 32887, 120, 65655, 32891, 127, 32898, 131, 32900, 32904, 140, 141, 147, 131219, 149, 151, 153, 158, 159, 65694, 32930, 32931, 131237, 167, 168, 169, 32937, 174, 175, 176, 177, 178, 32945, 65722, 32957, 191, 32961, 32963, 32968, 32970, 65739, 207, 32982, 65752, 217, 32987, 225, 226, 229, 32998, 32999, 33002, 236, 33006, 239, 65774, 242, 243, 255, 33028, 33033, 33036, 269, 33043, 33050, 65821, 33054, 288, 291, 292, 294, 33063, 33064, 302, 303, 33070, 305, 33078, 33079, 321, 33090, 323, 33091, 65864, 33103, 336, 33105, 339, 33114, 347, 33115, 131420, 33119, 353, 33121, 356, 33125, 33126, 370, 33146, 33150, 33158, 33159, 394, 395, 33167, 65942, 33180, 33182, 415, 417, 418, 33188, 33196, 431, 33204, 439, 33214, 33217, 33220, 454, 455, 456, 33232, 33234, 33